# Setup

### Import libs.

In [1]:
import os

import pyessv

### Set constants.

In [2]:
_VOCAB_DOMAIN = u"earth-system.modelling"
_VOCAB_SUBDOMAIN = u"ipsl.prodiguer"    
_VOCAB_IO_DIR = os.getenv("PRODIGUER_PYESSV_HOME")
_TERMSET_COMPUTE_NODE = u"compute_node"

# Partition creation

In [3]:
# Create vocab partition.
partition = pyessv.create_partition(
    _VOCAB_DOMAIN,
    _VOCAB_SUBDOMAIN,
    _VOCAB_IO_DIR
)

In [4]:
# Show partitions.
pyessv.show_partitions()

PYESSV PARTITION #1:
	domain = earth-system.modelling
	sub-domain = ipsl.prodiguer
	root-dir = /Users/macg/dev/tmp/pyessv-prodiguer


# Partition cache

### Load

In [5]:
# Loads terms from file system into memory.
partition.load()

2015-12-01T13:21:06.622705 [INFO] :: IPSL PYESSV > ** : partition cache loaded from @ /Users/macg/dev/tmp/pyessv-prodiguer


### Counts

In [6]:
# Cache count - all terms in all termsets;
assert partition.count() == 456

# Cache count - terms within a single termset;
assert partition.count(_TERMSET_COMPUTE_NODE) == 7

# Cache count - single term;
assert partition.count(_TERMSET_COMPUTE_NODE, "tgcc") == 1

### Access

In [7]:
# Cache access - all termsets;
termsets = partition.query()
assert isinstance(termsets, dict)

# Cache access - single termset;
termset = partition.query(_TERMSET_COMPUTE_NODE)
assert isinstance(termset, list)

# Cache access - single term.
term = partition.query(_TERMSET_COMPUTE_NODE, "tgcc")
assert isinstance(term, pyessv.Term)

### Listing

In [8]:
# Cache listing - all termsets;
partition.show()

termset --> earth-system.modelling:ipsl.prodiguer:model :: 26
termset --> earth-system.modelling:ipsl.prodiguer:compute_node :: 7
termset --> earth-system.modelling:ipsl.prodiguer:simulation_state :: 5
termset --> earth-system.modelling:ipsl.prodiguer:message_type :: 21
termset --> earth-system.modelling:ipsl.prodiguer:experiment_group :: 4
termset --> earth-system.modelling:ipsl.prodiguer:simulation_space :: 7
termset --> earth-system.modelling:ipsl.prodiguer:message_application :: 3
termset --> earth-system.modelling:ipsl.prodiguer:compute_node_login :: 122
termset --> earth-system.modelling:ipsl.prodiguer:job_type :: 3
termset --> earth-system.modelling:ipsl.prodiguer:model_forcing :: 18
termset --> earth-system.modelling:ipsl.prodiguer:compute_node_machine :: 10
termset --> earth-system.modelling:ipsl.prodiguer:institute :: 25
termset --> earth-system.modelling:ipsl.prodiguer:message_user :: 2
termset --> earth-system.modelling:ipsl.prodiguer:message_producer :: 2
termset --> earth

In [9]:
# Cache listing - single termset;
partition.show(_TERMSET_COMPUTE_NODE)

term --> earth-system.modelling:ipsl.prodiguer:compute_node::ccrt::[accepted]
term --> earth-system.modelling:ipsl.prodiguer:compute_node::cenaero::[accepted]
term --> earth-system.modelling:ipsl.prodiguer:compute_node::cines::[accepted]
term --> earth-system.modelling:ipsl.prodiguer:compute_node::idris::[accepted]
term --> earth-system.modelling:ipsl.prodiguer:compute_node::default::[accepted]
term --> earth-system.modelling:ipsl.prodiguer:compute_node::ipsl::[accepted]
term --> earth-system.modelling:ipsl.prodiguer:compute_node::tgcc::[accepted]


In [10]:
# Cache listing - single term;
partition.show(_TERMSET_COMPUTE_NODE, "tgcc")

term --> earth-system.modelling:ipsl.prodiguer:compute_node::tgcc::[accepted]


# Term management

### Creation

In [11]:
# Use partition to create new terms.
term = partition.create(_TERMSET_COMPUTE_NODE, u"xxx")
assert isinstance(term, pyessv.Term)

### Validation

In [12]:
# Invalid term - kind has invalid characters.
invalid_term = partition.create(u"&!@", u"xxx")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 1

invalid_term.show_errors()

term kind is invalid


In [13]:
# Invalid term - name has invalid characters.
invalid_term = partition.create(_TERMSET_COMPUTE_NODE, u"x&!")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 1

invalid_term.show_errors()

term name is invalid


In [14]:
# Invalid term - kind & name have invalid characters.
invalid_term = partition.create(u"&!@", u"x&!")

assert invalid_term.is_valid == False
assert len(invalid_term.errors) == 2

invalid_term.show_errors()

term kind is invalid
term name is invalid


### Encoding

In [15]:
# Encode as python dictionary.
as_dict = term.as_dict()
assert isinstance(as_dict, dict)

# Encode as JSON.
as_json = term.as_json()
assert isinstance(as_json, str)

### Persistence

In [16]:
# Saving a term writes to cache & file system.
assert term.is_cached == False
assert term.is_written == False

term.save()

assert term.is_cached == True
assert term.is_written == True

2015-12-01T13:21:07.114497 [INFO] :: IPSL PYESSV > ** : term written to file system: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[pending]
2015-12-01T13:21:07.114558 [INFO] :: IPSL PYESSV > ** : term written to cache: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[pending]


# Term Governance

In [17]:
# What is initial governance status ?
term.status

u'pending'

In [18]:
# Accept.
term.accept()

2015-12-01T13:21:07.128113 [INFO] :: IPSL PYESSV > ** : term written to file system: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[accepted]


In [19]:
# Reject.
term.reject()

2015-12-01T13:21:07.131947 [INFO] :: IPSL PYESSV > ** : term written to file system: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[rejected]


In [20]:
# Deprecate.
term.deprecate()

2015-12-01T13:21:07.135958 [INFO] :: IPSL PYESSV > ** : term written to file system: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[obsolete]


In [21]:
# Destroy.
assert partition.count(_TERMSET_COMPUTE_NODE) == 8
assert partition.query(_TERMSET_COMPUTE_NODE, "xxx") is not None

term.destroy()

assert partition.count(_TERMSET_COMPUTE_NODE) == 7
assert partition.query(_TERMSET_COMPUTE_NODE, "xxx") is None

2015-12-01T13:21:07.141281 [INFO] :: IPSL PYESSV > ** : term removed from file system: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[obsolete]
2015-12-01T13:21:07.141406 [INFO] :: IPSL PYESSV > ** : term removed from cache: term --> earth-system.modelling:ipsl.prodiguer:compute_node::xxx::[obsolete]


# Term Parsing

In [22]:
# Parse a name that matches. 
name = "tgcc"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, name)
assert name == parsed_name

In [23]:
# Parse a name that almost matches. 
name = "tGcC"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, name)
assert name != parsed_name
assert parsed_name == "tgcc"

In [24]:
# Parse an alternative name.
alt_name = "saclay"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, alt_name)
assert alt_name != parsed_name
assert parsed_name == "tgcc"

In [25]:
# Parse an alias.
alias = "TGCC-1"
parsed_name = partition.parse(_TERMSET_COMPUTE_NODE, alias)
assert alias != parsed_name
assert parsed_name == "tgcc"

# Term Associations

In [26]:
# TODO !!!

# Term Grouping

In [28]:
# TODO 
# Write groups of terms to file system in blocks so as to streamline downloading in UI !!!